In [ ]:
from ultralytics import YOLO
from pathlib import Path
from datetime import datetime
import torch
import time

# Path to your YOLO model
MODEL_CONFIG = {"conf": 0.2, "path": Path("best.pt")}  # Assuming your weights are in the same directory

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def find_largest_bbox_label(bboxes):
    largest_bbox_area = 0.0
    largest_bbox_label = None

    for bbox in bboxes:
        label = bbox['label']
        _, _, width, height = bbox['xywh']
        bbox_area = width * height
        if bbox_area > largest_bbox_area:
            largest_bbox_area = bbox_area
            largest_bbox_label = label

    return largest_bbox_label, largest_bbox_area


def image_inference(image_path, obs_id, image_counter):
    formatted_time = datetime.fromtimestamp(time.time()).strftime('%d-%m_%H-%M-%S.%f')[:-3]
    img_name = f"img_{formatted_time}"

    model = YOLO(MODEL_CONFIG["path"])
    model.to(device)

    results = model.predict(
        source=image_path,
        save=True,
        conf=MODEL_CONFIG["conf"],
        imgsz=640,
        device=device
    )

    bboxes = []
    for r in results:
        for c in r:
            # Get the predicted class (label)
            label = c.names[c.boxes.cls.tolist().pop()]
            print(f"Predicted Class: {label}")  # Printing the predicted class

            # Store bbox details
            bboxes.append({"label": label, "xywh": c.boxes.xywh.tolist().pop()})

    largest_bbox_label, largest_bbox_area = find_largest_bbox_label(bboxes)

    if largest_bbox_label is None:
        print("No objects detected.")
        return None

    name_of_image = f"task_obs_id_{obs_id}_{image_counter}.jpg"

    return {
        "type": "IMAGE_RESULTS",
        "data": {
            "obs_id": obs_id,
            "img_id": largest_bbox_label,
            "bbox_area": largest_bbox_area
        },
        "image_path": Path("captured_images") / img_name / name_of_image
    }


# Example Test with IMG_0762.jpg
image = "test4.jpg"
result = image_inference(image, "01", 1)



image 1/1 C:\Users\jianh\Desktop\SC2079 MDP\augmented_110207_IMG_2368.jpg: 640x640 1 left, 284.5ms
Speed: 7.6ms preprocess, 284.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to C:\Users\jianh\runs\detect\predict4
Predicted Class: left


In [ ]:
from ultralytics import YOLO
from pathlib import Path
from datetime import datetime
import torch
import time
import os

# Path to your YOLO model
MODEL_CONFIG = {"conf": 0.4, "path": Path("best.pt")}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

def find_largest_bbox_label(bboxes):
    largest_bbox_area = 0.0
    largest_bbox_label = None

    for bbox in bboxes:
        label = bbox['label']
        _, _, width, height = bbox['xywh']
        bbox_area = width * height
        if bbox_area > largest_bbox_area:
            largest_bbox_area = bbox_area
            largest_bbox_label = label

    return largest_bbox_label, largest_bbox_area

def image_inference(model, image_path, obs_id, image_counter, output_dir):
    formatted_time = datetime.now().strftime('%d-%m_%H-%M-%S.%f')[:-3]
    img_name = f"task_obs_id_{obs_id}_{image_counter}.jpg"

    results = model.predict(
        source=image_path,
        save=True,
        conf=MODEL_CONFIG["conf"],
        imgsz=640,
        device=device
    )

    # Save labeled image to the common output directory
    labeled_img_path = Path(results[0].save_dir) / image_path.name
    output_file_path = output_dir / img_name

    os.makedirs(output_dir, exist_ok=True)
    labeled_img_path.rename(output_file_path)  # Move labeled image

    bboxes = []
    for result in results:
        for box in result.boxes:
            cls_index = int(box.cls.tolist()[0])
            label = result.names[cls_index]
            xywh = box.xywh.tolist()[0]
            bboxes.append({"label": label, "xywh": xywh})

    largest_bbox_label, largest_bbox_area = find_largest_bbox_label(bboxes)

    if largest_bbox_label:
        print(f"Image '{image_path.name}': Detected '{largest_bbox_label}' with bbox area {largest_bbox_area:.2f}")
    else:
        print(f"Image '{image_path.name}': No objects detected.")
        
# Load the model once
model = YOLO(MODEL_CONFIG["path"])
model.to(device)

# Define paths
test_folder = Path("test_images")
output_dir = Path("results_images")

# Process images
image_counter = 1
for image_file in test_folder.glob("*.*"):
    if image_file.suffix.lower() in ['.jpg', '.jpeg', '.png']:
        result = image_inference(model, image_file, "01", image_counter, output_dir)
        image_counter += 1
